<h1 align="center"><font size="5">COLLABORATIVE FILTERING</font></h1>

In [2]:
import pandas as pd
from math import sqrt
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

<hr>

<a id="ref3"></a>
# Collaborative Filtering

El filtrado colaborativo es una técnica que puede filtrar elementos que podrían gustarle a un usuario en función de las 
reacciones de usuarios similares. Funciona buscando en un gran grupo de personas y encontrando un grupo más pequeño
de usuarios con gustos similares a un usuario en particular.

Lectura dataframe ratings_df, news_df y base_clientes

In [3]:
ratings_df = pd.read_csv(r'C:\Users\maria\Desktop\MasterBD\TFM\datos\ratings_dfS.csv',encoding='utf-8-sig', sep=';')

In [6]:
ratings_df

,Unnamed: 0,Unnamed: 0.1,user_id,new_id,rating,suscr_new
0,0,0,cl_0,gen_2_230720,1.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
1,1,1,cl_0,gen_4_230720,3.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
2,2,2,cl_0,gen_5_230720,3.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
3,3,3,cl_0,gen_6_230720,1.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
4,4,4,cl_0,gen_7_230720,4.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
...,...,...,...,...,...,...
68471181,85582225,85582225,cl_44999,dep_688_300720,2.0,"['beta', 'delta', 'omega']"
68471182,85582226,85582226,cl_44999,dep_694_300720,1.0,"['beta', 'delta', 'omega']"
68471183,85582228,85582228,cl_44999,dep_707_300720,2.0,"['beta', 'delta', 'omega']"
68471184,85582229,85582229,cl_44999,eco_774_300720,3.0,"['gamma', 'delta', 'omega']"


In [7]:
news_df = pd.read_csv(r'C:\Users\maria\Desktop\MasterBD\TFM\datos\news_df.csv',encoding='utf-8-sig', sep=';')

In [8]:
ratings_df.drop(ratings_df.columns[ratings_df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [16]:
news_df.drop(news_df.columns[news_df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)


In [9]:
base_clientes = pd.read_csv(r'C:\Users\maria\Desktop\MasterBD\TFM\datos\base_clientes.csv',encoding='utf-8-sig', sep=';')

In [8]:
base_clientes.head()

,Unnamed: 0,Customer_ID,noticias_leidas,noticias_puntuadas,puntuacion_media,Web,suscripcion
0,0,1,371.0,284,4.073684,El Confidencial,alpha
1,1,2,372.0,300,3.956811,El Confidencial,alpha
2,2,3,364.0,289,4.155172,El Confidencial,alpha
3,3,4,364.0,305,4.062092,El Confidencial,alpha
4,4,5,368.0,294,4.054237,El Confidencial,alpha


Dataframe user_id suscripcion

In [10]:
cliente_suscripcion = base_clientes.filter(['Customer_ID','suscripcion'], axis=1)

In [11]:
cliente_suscripcion = cliente_suscripcion.drop_duplicates()

In [12]:
cliente_suscripcion

,Customer_ID,suscripcion
0,1,alpha
1,2,alpha
2,3,alpha
3,4,alpha
4,5,alpha
...,...,...
44995,44996,omega
44996,44997,omega
44997,44998,omega
44998,44999,omega


In [13]:
customer_id = []
for i in cliente_suscripcion['Customer_ID']:
    clie_id = 'cl_'+ str(i) 
    customer_id.append(clie_id)
    

In [14]:
cliente_suscripcion['Customer_ID']=customer_id

In [15]:
cliente_suscripcion

,Customer_ID,suscripcion
0,cl_1,alpha
1,cl_2,alpha
2,cl_3,alpha
3,cl_4,alpha
4,cl_5,alpha
...,...,...
44995,cl_44996,omega
44996,cl_44997,omega
44997,cl_44998,omega
44998,cl_44999,omega


Lectura main

In [16]:
main = pd.read_csv(r'C:\Users\maria\Desktop\MasterBD\TFM\datos\main.csv',encoding='utf-8-sig', sep=';')

In [16]:
main.head(3)

,Unnamed: 0,new_id,contador,Web,Título,Autor,tipo_suscripcion,Cabecera,Fecha,rating_medio,total_lecturas
0,0,gen_2_230720,1,El Confidencial,Detienen al director de una residencia por dro...,Europa Press,gen,0,2020-07-23,2.990605,22467.0
1,2,gen_3_230720,1,El Confidencial,El Supremo revoca los permisos de Forcadell y ...,Pablo Gabilondo,gen,0,2020-07-23,2.998222,22598.0
2,4,gen_4_230720,1,El Confidencial,El 'efecto mariposa' del coronavirus: solo los...,Alberto Pérez Giménez,gen,0,2020-07-23,3.016924,22418.0


Se crea Dataframe con new_id título suscripcion

In [17]:
noticia_suscripcion = main.filter(['new_id','Título','tipo_suscripcion'], axis=1)

In [14]:
noticia_suscripcion

,new_id,Título,tipo_suscripcion
0,gen_2_230720,Detienen al director de una residencia por dro...,gen
1,gen_3_230720,El Supremo revoca los permisos de Forcadell y ...,gen
2,gen_4_230720,El 'efecto mariposa' del coronavirus: solo los...,gen
3,gen_5_230720,Andalucía y Galicia se rebelan contra el fondo...,gen
4,gen_6_230720,Los aliados del Gobierno dan un aviso para los...,gen
...,...,...,...
4637,eco_802_300720,"Acciona recorta un 85,7% su beneficio por el C...",eco
4638,eco_803_300720,"Samsung ganó un 7,33% más en el segundo trimes...",eco
4639,eco_805_300720,Aena abre la puerta a retrasar los crecimiento...,eco
4640,eco_806_300720,El Gobierno de Madrid da marcha atrás en dos d...,eco


Seleccionamos un usuario al que recomendaremos noticias. En este caso se selecciona uno como muestra, pero el sistema se aplicaría a todos los usuarios dentro de iteración o input().

Obtenemos el tipo de suscripcion, las noticias leídas y el rating dado por el usuario al que se va a recomendar.

In [18]:
# Recordar que el id se pasara como parámetro
susc_cl = cliente_suscripcion.loc[cliente_suscripcion['Customer_ID']=='cl_47']

In [19]:
tipo_suscripcion = susc_cl['suscripcion']
tipo_suscripcion = tipo_suscripcion.values[0]

In [18]:
tipo_suscripcion

'alpha'

Se crea diciionario con las modalidades de suscripción

In [20]:
modalidad_suscr = {'alpha':['gen'],'beta':['gen','dep'],'gamma':['gen','eco'],'delta':['gen','dep','eco'],'omega':['gen','dep','eco']}

In [21]:
suscripcion = modalidad_suscr.get(tipo_suscripcion) 

El  cliente tiene acceso a las noticias gen

In [30]:
suscripcion

['gen']

Noticias leídas por el usuario Input

In [22]:
userInput = ratings_df.loc[ratings_df['user_id']=='cl_47','new_id':'rating']

In [33]:
userInput

,new_id,rating
86549,gen_2_230720,1.0
86550,gen_3_230720,3.0
86551,gen_5_230720,4.0
86552,gen_6_230720,1.0
86553,gen_7_230720,3.0
...,...,...
88349,gen_565_300720,2.0
88350,gen_566_300720,5.0
88351,gen_567_300720,2.0
88352,gen_568_300720,3.0


In [23]:
inputNews = pd.DataFrame(userInput)

In [35]:
inputNews

,new_id,rating
86549,gen_2_230720,1.0
86550,gen_3_230720,3.0
86551,gen_5_230720,4.0
86552,gen_6_230720,1.0
86553,gen_7_230720,3.0
...,...,...
88349,gen_565_300720,2.0
88350,gen_566_300720,5.0
88351,gen_567_300720,2.0
88352,gen_568_300720,3.0


Añadimos el título de la noticia 

In [24]:
#Filtrado por news_id en news_df
titulo = news_df[news_df['new_id'].isin(inputNews['new_id'].tolist())]


In [25]:
#Merge
inputNews = pd.merge(titulo, inputNews)


Obtenemos los usuarios-noticias que han visto las mismas noticias puntuadas  por el usuaro "input" y que son accesibles con sususcripcion.

In [26]:
userSubsetP = ratings_df[ratings_df['new_id'].isin(inputNews['new_id'].tolist())]

In [52]:
userSubsetP

,user_id,new_id,rating,suscr_new
0,cl_0,gen_2_230720,1.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
2,cl_0,gen_5_230720,3.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
3,cl_0,gen_6_230720,1.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
4,cl_0,gen_7_230720,4.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
5,cl_0,gen_11_230720,2.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
...,...,...,...,...
68471170,cl_44999,gen_466_300720,4.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
68471171,cl_44999,gen_486_300720,5.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
68471173,cl_44999,gen_525_300720,2.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
68471174,cl_44999,gen_530_300720,5.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"


In [53]:
#userSubsetP = userSubsetP[userSubsetP.apply(lambda x: tipo_suscripcion in x['suscr_new'], axis=1)]


In [27]:
userSubsetP.suscr_new.unique()

array(["['alpha', 'beta', 'gamma', 'delta', 'omega']"], dtype=object)

Agrupacion de filas por user_id

In [28]:
userSubsetPGroup = userSubsetP.groupby(['user_id'])

In [58]:
userSubsetPGroup.user_id.unique()

user_id
cl_0          [cl_0]
cl_1          [cl_1]
cl_10        [cl_10]
cl_100      [cl_100]
cl_1000    [cl_1000]
             ...    
cl_9995    [cl_9995]
cl_9996    [cl_9996]
cl_9997    [cl_9997]
cl_9998    [cl_9998]
cl_9999    [cl_9999]
Name: user_id, Length: 45000, dtype: object

In [59]:
#Ejemplo de un usuario que coincide en lectura con el input user
userSubsetPGroup.get_group('cl_9999')


,user_id,new_id,rating,suscr_new
11637303,cl_9999,gen_2_230720,5.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
11637304,cl_9999,gen_3_230720,5.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
11637305,cl_9999,gen_5_230720,1.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
11637306,cl_9999,gen_6_230720,1.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
11637307,cl_9999,gen_7_230720,3.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
...,...,...,...,...
11639680,cl_9999,gen_530_300720,5.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
11639681,cl_9999,gen_545_300720,2.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
11639683,cl_9999,gen_551_300720,1.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
11639686,cl_9999,gen_560_300720,2.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"


Ordenamos de forma que los usuarios que compartan más noticias en común con el Input User tengan mayor prioridad. De esta forma para la recomendación no recorreremos todos los usuarios, si no los que tengan más noticias en común.

In [29]:
userSubsetPGroup = sorted(userSubsetPGroup, key = lambda x: len(x[1]), reverse= True)

Vemos los usuarios más afines a Input User en cuanto a noticias leídas. El primero va a ser el mismo usuario, que no se tendrá en cuenta.

In [61]:
userSubsetPGroup[0:3]

[('cl_47',
        user_id          new_id  rating  \
  86549   cl_47    gen_2_230720     1.0   
  86550   cl_47    gen_3_230720     3.0   
  86551   cl_47    gen_5_230720     4.0   
  86552   cl_47    gen_6_230720     1.0   
  86553   cl_47    gen_7_230720     3.0   
  ...       ...             ...     ...   
  88349   cl_47  gen_565_300720     2.0   
  88350   cl_47  gen_566_300720     5.0   
  88351   cl_47  gen_567_300720     2.0   
  88352   cl_47  gen_568_300720     3.0   
  88353   cl_47  gen_569_300720     2.0   
  
                                            suscr_new  
  86549  ['alpha', 'beta', 'gamma', 'delta', 'omega']  
  86550  ['alpha', 'beta', 'gamma', 'delta', 'omega']  
  86551  ['alpha', 'beta', 'gamma', 'delta', 'omega']  
  86552  ['alpha', 'beta', 'gamma', 'delta', 'omega']  
  86553  ['alpha', 'beta', 'gamma', 'delta', 'omega']  
  ...                                             ...  
  88349  ['alpha', 'beta', 'gamma', 'delta', 'omega']  
  88350  ['alpha', 'be

#### Semejanza de los usuarios al Input User
Se va a comparar a todos los usuarios (acabaremos usando una muestra) al Input user para encontrar el de mayor similitud. Esto se va a realizar a través del Coeficiente de Correlacón de Pearson. Se utiliza para medir la fuerza de una asociación lineal entre dos variables.

Se va a seleccionar una muestra de los usuarios sobre los que se iterará. Esto se hace para acortar tiempos , pero se puede realizar con todos los usuarios.

In [30]:
# No seleccionamos desde el primer usuario ya que es el input user
userSubsetPGroup = userSubsetPGroup[1:100]

Calculamos la Correlación de Pearson entre el Input User y los usurios seleccionados. Guardamos el resultado en un diccionario, donde la clave será el user_id y el valor el coeficiente.

In [63]:
userSubsetPGroup

[('cl_337',
         user_id          new_id  rating  \
  618704  cl_337    gen_2_230720     2.0   
  618705  cl_337    gen_3_230720     3.0   
  618706  cl_337    gen_6_230720     4.0   
  618707  cl_337   gen_10_230720     4.0   
  618708  cl_337   gen_12_230720     2.0   
  ...        ...             ...     ...   
  620602  cl_337  gen_560_300720     3.0   
  620603  cl_337  gen_565_300720     2.0   
  620604  cl_337  gen_566_300720     2.0   
  620605  cl_337  gen_568_300720     1.0   
  620606  cl_337  gen_569_300720     2.0   
  
                                             suscr_new  
  618704  ['alpha', 'beta', 'gamma', 'delta', 'omega']  
  618705  ['alpha', 'beta', 'gamma', 'delta', 'omega']  
  618706  ['alpha', 'beta', 'gamma', 'delta', 'omega']  
  618707  ['alpha', 'beta', 'gamma', 'delta', 'omega']  
  618708  ['alpha', 'beta', 'gamma', 'delta', 'omega']  
  ...                                              ...  
  620602  ['alpha', 'beta', 'gamma', 'delta', 'omega']  
 

In [31]:
# Creamos el diccionario
pearsonCorrelationDict = {}

In [32]:
#Para cada grupo de usuiarios
for name, group in userSubsetPGroup:
    
    #Ordenamos por new_id en input y el grupo
    group = group.sort_values(by='new_id')
    inputNews = inputNews.sort_values(by='new_id')
    
    #Longitud de cada grupo
    nRatings = len(group)
    #Ratings de las pelis que tienen en común
    temp_df = inputNews[inputNews['new_id'].isin(group['new_id'].tolist())]
    #Se guardan en una lista los ratings de los usuarios
    tempRatingList = temp_df['rating'].tolist()
    #Se guardan en una lista los ratings del Inout User
    tempGroupList = group['rating'].tolist()
    #Correlación de Pearson 
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    
    # Si el denominador es distinto de 0, divide, else, correlación 0.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


In [33]:
pearsonCorrelationDict.items()

dict_items([('cl_337', 0.012921955378536841), ('cl_27893', -0.02722054930593858), ('cl_9301', -0.016360678669461455), ('cl_141', 0.0070260831272467945), ('cl_36827', -0.008400572530827929), ('cl_36819', -0.005393299178612076), ('cl_36953', -0.018089053106988107), ('cl_175', 0.005772521986305677), ('cl_36226', -0.03520603524212024), ('cl_9792', 0.0023828784986125195), ('cl_18505', -0.04206663471840689), ('cl_9333', -0.03770638025515663), ('cl_934', -0.007883481051032004), ('cl_199', -0.0030740523114213747), ('cl_36599', -0.046846953901175256), ('cl_36960', -0.00046814819809061237), ('cl_36367', -0.001978785331132451), ('cl_36388', -0.05782792482403524), ('cl_36572', -0.04246774561047224), ('cl_9125', 0.0008509345751844085), ('cl_9713', -0.020253840849439052), ('cl_18075', 0.0392578253293114), ('cl_18133', -0.015853997533465395), ('cl_27814', -0.011613375167079299), ('cl_36058', -0.013272060464857535), ('cl_18750', -0.0263292103218812), ('cl_27305', -0.0012931595564056163), ('cl_36141', 

In [35]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers = topUsers.drop([0], axis=0)
topUsers.head()

,similarityIndex,user_id
73,0.055811,cl_9377
66,0.048481,cl_9344
32,0.044036,cl_18605
64,0.042784,cl_315
21,0.039258,cl_18075


In [36]:
topUsers.to_csv(r'C:\Users\maria\Desktop\MasterBD\TFM\datos\topUsers.csv',sep=';',encoding='utf-8-sig', index = True)

A partir de aqui se repetirán los procesos cambiando el merge con ratings de forma que se obtengan las recomendaciones en base a la suscripcion. (Noticias según suscripción y noticias recomendas a las que no se tiene acceso con esa suscripción)

Merge con todo ratings sin importar el tipo suscripción

In [37]:
topUsersRatingT=topUsers.merge(ratings_df, left_on='user_id', right_on='user_id', how='inner')


In [48]:
topUsersRatingT.to_csv(r'C:\Users\maria\Desktop\MasterBD\TFM\datos\topUsersRatingT.csv',sep=';',encoding='utf-8-sig', index = True)

Merge con todo ratings con mismo tipo suscripción que el In put User

In [41]:
topUsersRatingS=topUsers.merge(ratings_df[ratings_df.apply(lambda x: tipo_suscripcion in x['suscr_new'], axis=1)], left_on='user_id', right_on='user_id', how='inner')


In [49]:
topUsersRatingT.to_csv(r'C:\Users\maria\Desktop\MasterBD\TFM\datos\topUsersRatingS.csv',sep=';',encoding='utf-8-sig', index = True)

In [42]:
c

array(["['alpha', 'beta', 'gamma', 'delta', 'omega']",
       "['beta', 'delta', 'omega']", "['gamma', 'delta', 'omega']"],
      dtype=object)

In [43]:
topUsersRatingS.suscr_new.unique()

array(["['alpha', 'beta', 'gamma', 'delta', 'omega']"], dtype=object)

In [44]:
topUsersRatingT

,similarityIndex,user_id,new_id,rating,suscr_new
0,0.055811,cl_9377,gen_2_230720,5.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
1,0.055811,cl_9377,gen_4_230720,1.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
2,0.055811,cl_9377,gen_5_230720,3.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
3,0.055811,cl_9377,gen_7_230720,5.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
4,0.055811,cl_9377,gen_10_230720,3.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
...,...,...,...,...,...
131761,-0.003661,cl_9491,dep_713_300720,4.0,"['beta', 'delta', 'omega']"
131762,-0.003661,cl_9491,dep_715_300720,4.0,"['beta', 'delta', 'omega']"
131763,-0.003661,cl_9491,dep_717_300720,2.0,"['beta', 'delta', 'omega']"
131764,-0.003661,cl_9491,dep_718_300720,4.0,"['beta', 'delta', 'omega']"


In [47]:
topUsersRatingS

,similarityIndex,user_id,new_id,rating,suscr_new
0,0.055811,cl_9377,gen_2_230720,5.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
1,0.055811,cl_9377,gen_4_230720,1.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
2,0.055811,cl_9377,gen_5_230720,3.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
3,0.055811,cl_9377,gen_7_230720,5.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
4,0.055811,cl_9377,gen_10_230720,3.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
...,...,...,...,...,...
91930,-0.003661,cl_9491,gen_555_300720,3.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
91931,-0.003661,cl_9491,gen_565_300720,2.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
91932,-0.003661,cl_9491,gen_567_300720,5.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"
91933,-0.003661,cl_9491,gen_568_300720,1.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']"


In [50]:
# Multiplicar la semejanza por el rating del usuario.
topUsersRatingT['weightedRating'] = topUsersRatingT['similarityIndex']*topUsersRatingT['rating']
topUsersRatingT.head()

,similarityIndex,user_id,new_id,rating,suscr_new,weightedRating
0,0.055811,cl_9377,gen_2_230720,5.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']",0.279053
1,0.055811,cl_9377,gen_4_230720,1.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']",0.055811
2,0.055811,cl_9377,gen_5_230720,3.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']",0.167432
3,0.055811,cl_9377,gen_7_230720,5.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']",0.279053
4,0.055811,cl_9377,gen_10_230720,3.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']",0.167432


In [51]:
# Multiplicar la semejanza por el rating del usuario.
topUsersRatingS['weightedRating'] = topUsersRatingS['similarityIndex']*topUsersRatingS['rating']
topUsersRatingS.head()

,similarityIndex,user_id,new_id,rating,suscr_new,weightedRating
0,0.055811,cl_9377,gen_2_230720,5.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']",0.279053
1,0.055811,cl_9377,gen_4_230720,1.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']",0.055811
2,0.055811,cl_9377,gen_5_230720,3.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']",0.167432
3,0.055811,cl_9377,gen_7_230720,5.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']",0.279053
4,0.055811,cl_9377,gen_10_230720,3.0,"['alpha', 'beta', 'gamma', 'delta', 'omega']",0.167432


In [52]:
#Aplica una suma a los usuarios principales después de agruparlo por userId
tempTopUsersRatingT = topUsersRatingT.groupby('new_id').sum()[['similarityIndex','weightedRating']]
tempTopUsersRatingT.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRatingT.head()

,sum_similarityIndex,sum_weightedRating
new_id,,
dep_370_250720,0.280945,0.919774
dep_371_250720,0.325111,1.023351
dep_372_250720,0.292540,1.019379
dep_372_260720,0.372300,1.189650
dep_373_250720,0.365453,0.804280


In [53]:
#Aplica una suma a los usuarios principales después de agruparlo por userId
tempTopUsersRatingS = topUsersRatingS.groupby('new_id').sum()[['similarityIndex','weightedRating']]
tempTopUsersRatingS.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRatingS.head()

,sum_similarityIndex,sum_weightedRating
new_id,,
gen_100_230720,0.592203,1.652045
gen_100_250720,0.629539,1.853809
gen_100_260720,0.553547,1.534943
gen_100_280720,0.570284,1.887632
gen_100_290720,0.548644,1.880412


In [54]:
#Dataframe vacío
recommendation_dfT = pd.DataFrame()
#Cogemos el promedio ponderado
recommendation_dfT['weighted average recommendation score'] = tempTopUsersRatingT['sum_weightedRating']/tempTopUsersRatingT['sum_similarityIndex']
recommendation_dfT['new_id'] = tempTopUsersRatingT.index
recommendation_dfT.head()

,weighted average recommendation score,new_id
new_id,,
dep_370_250720,3.273861,dep_370_250720
dep_371_250720,3.147700,dep_371_250720
dep_372_250720,3.484581,dep_372_250720
dep_372_260720,3.195404,dep_372_260720
dep_373_250720,2.200778,dep_373_250720


In [55]:
#Dataframe vacío
recommendation_dfS = pd.DataFrame()
#Cogemos el promedio ponderado
recommendation_dfS['weighted average recommendation score'] = tempTopUsersRatingS['sum_weightedRating']/tempTopUsersRatingS['sum_similarityIndex']
recommendation_dfS['new_id'] = tempTopUsersRatingS.index
recommendation_dfS.head()

,weighted average recommendation score,new_id
new_id,,
gen_100_230720,2.789660,gen_100_230720
gen_100_250720,2.944708,gen_100_250720
gen_100_260720,2.772921,gen_100_260720
gen_100_280720,3.309987,gen_100_280720
gen_100_290720,3.427379,gen_100_290720


Se ordena para ver el top

In [56]:
recommendation_dfT = recommendation_dfT.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_dfT.head(10)

,weighted average recommendation score,new_id
new_id,,
eco_567_250720,4.532826,eco_567_250720
dep_898_230720,4.497295,dep_898_230720
eco_343_290720,4.304327,eco_343_290720
eco_582_260720,4.244078,eco_582_260720
eco_800_260720,4.228426,eco_800_260720
dep_712_280720,4.220884,dep_712_280720
eco_550_230720,4.194817,eco_550_230720
eco_341_280720,4.184803,eco_341_280720
eco_303_290720,4.180472,eco_303_290720


In [57]:
recommendation_dfS = recommendation_dfS.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_dfS.head(10)

,weighted average recommendation score,new_id
new_id,,
gen_486_300720,4.145582,gen_486_300720
gen_446_260720,4.121294,gen_446_260720
gen_499_250720,4.084284,gen_499_250720
gen_95_260720,4.080603,gen_95_260720
gen_454_230720,4.013410,gen_454_230720
gen_160_250720,3.991519,gen_160_250720
gen_273_260720,3.965933,gen_273_260720
gen_472_260720,3.958738,gen_472_260720
gen_355_230720,3.925097,gen_355_230720


Top de noticias recomendadas para el Input user accesibles con otro tipo de suscripción

In [58]:
news_df.loc[news_df['new_id'].isin(recommendation_dfT.head(10)['new_id'].tolist())]

,Unnamed: 0,new_id,Título
224,224,eco_550_230720,"16,0%"
569,569,eco_567_250720,El Gobierno autoriza la central de Alamaraz ha...
698,698,eco_582_260720,"El bono español deja ganancias del 7,8% desde ..."
1550,1550,dep_898_230720,"2,20 de altura, hijo de una leyenda y se salió..."
2671,2671,dep_383_260720,Foto familiar de los Messi...
2835,2835,eco_800_260720,MBA que aprenden la lección del Covid-19
3434,3434,eco_341_280720,El valor de los activos de Prologis se eleva a...
3600,3600,dep_712_280720,"Keidi Baré, liquidez al Atlético"
3850,3850,eco_303_290720,Batalla entre el sector y la AIReF por el ince...
3885,3885,eco_343_290720,La Fed dará pistas de los cambios que llegarán...


Top de noticias recomendadas para el Input user accesibles con el tipo de suscripción del Input User

In [59]:
news_df.loc[news_df['new_id'].isin(recommendation_dfS.head(10)['new_id'].tolist())]

,Unnamed: 0,new_id,Título
151,151,gen_355_230720,Coronavirus: las imágenes de la pandemia en el...
175,175,gen_454_230720,Lo que no se vio en la final de «MasterChef»: ...
2254,2254,gen_160_250720,"La salud mental de Kanye West, un candidato a ..."
2416,2416,gen_499_250720,"Los testimonios de un superviviente, el famili..."
2540,2540,gen_95_260720,La indiferencia ante el infierno
2643,2643,gen_273_260720,Fotogalería: Citroën ZX Rally Raid
2710,2710,gen_446_260720,"China, la baza de Trump para la reelección: “E..."
2721,2721,gen_472_260720,¡Hola! Somos Viviendea
4323,4323,gen_261_300720,Iker Jiménez recupera «La estirpe de los libre...
4439,4439,gen_486_300720,Un químico bate el récord en memorizar decima...
